# API REST : accéder à des données et des fonctionnalités via le web


Gabriel Couture, Félix-Antoine Fortin, Olivier Chouinard-Banville

In [6]:
import io

import pydicom
import matplotlib.pyplot as plt
from pydicom import uid
from pydicom.filebase import DicomFileLike
from matplotlib.image import imread


def read_dicom_dataset(raw_data: bytes) -> pydicom.Dataset:
    """Permet de lire les bytes d'un fichier DICOM et le transformer en Dataset DICOM"""
    instance_uid = uid.generate_uid()
    buffer = io.BytesIO(raw_data)
    ds = pydicom.dcmread(buffer)

    return ds


def dicom_dataset_to_bytes(ds: pydicom.Dataset) -> io.BytesIO:
    """Permet d'écrire un Dataset DICOM en bytes"""
    buffer = io.BytesIO()
    memory_dataset = DicomFileLike(buffer)
    pydicom.dcmwrite(memory_dataset, ds)
    memory_dataset.seek(0)

    return buffer


def show_image_from_bytes(raw_data: bytes) -> None:
    buffer = io.BytesIO(raw_data)
    img = imread(buffer, format='png')

    plt.figure(figsize=(16, 12))
    plt.imshow(img)
    plt.axis('off')
    plt.show()


URL = 'http://172.17.0.2:5000'

## Mise en situation

- Vous êtes un stagiaire en physique médiale
- Vous devez évaluer la qualité de traitements de radiothérapie
- Pour ce faire : nous utilisons les histogrammes dose-volume (DVH)


## Radiothérapie : cours express

<center><img src="./imgs/flot-radiotherapie.png" style="width: 700px;"></center>
<br>
<center><img src="./imgs/radiotherapy.png" style="width: 400px;"></center>


## Radiothérapie : images
Acquisition d'images (ex. CT)
<br>
<center><img src="./imgs/phantom-thorax.png" style="width: 300px;"></center>


## Radiothérapie : contours
- Tracé de structure (organes/région d'intérêt)

<table border="0">
 <tr>
    <td><img src="./imgs/contours.png" style="height: 250px;"></td>
    <td><img src="./imgs/structure.png" style="height: 250px;"></td>
 </tr>
</table>

    
## Radiothérapie : cours express
- Planification de la dose

<center><img src="./imgs/dose-distribution.png" style="height: 200px;"></center>

## Radiothérapie : cours express
- Données impliquées
    - Images CT
    - Structure (région/organe dessiné)
    - Carte de dose (array en 3D de la dose délivrée)
<center><img src="./imgs/ct-struct-dose.gif" style="width: 100%;"></center>


## Comment évaluer la qualité du traitement?

- Construire un histogramme dose-volume (DVH)!

<center><img src="./imgs/dvh-process.png" style="height: 300px;"></center>
<center><img src="./imgs/dvh-diff-to-cumulative.png" style="height: 250px;"></center>

## Résultat final : un DVH!
<center><img src="./imgs/dvh-exemple.png" style="width: 550px;"></center>

# Atelier : ce qui est à faire

- Une API REST vous est mise à disposition
    - Elle expose des données, des fonctions de calcul de DVH et plus!

<center><img src="./imgs/atelier.png" style="width: 1200px;"></center>

## Données à manipuler

- 3 fichiers DICOM RTStruct (cylindre, shpère, cone)
- 1 fichier DICOM RTDose
<table border="0">
 <tr>
    <td><img src="./imgs/cylinder.png" style="height: 250px;"></td>
    <td><img src="./imgs/sphere.png" style="height: 250px;"></td>
    <td><img src="./imgs/cone.png" style="height: 250px;"></td>
    <td><img src="./imgs/dose-workflow.png" style="height: 250px;"></td>
 </tr>
</table>
Source : https://doi.org/10.1118/1.4923175


# Informations utiles
### Librairie Python `requests`
 
```python
requests.get(url='http://<url>/<route>', params={'key': 'value'})

requests.post(url='http://<url>/<route>',
              data={'key': 'value'},          # Pour les valeurs (str, float ou int)
              files={'key': file_bytes},      # Pour des fichiers (sous forme de bytes)
              json={'key': {'key': 'value'}}) # Pour des dictionnaires ou listes

requests.put(url='http://<url>/<route>',
             data={'key': 'value'},           # ...
             files={'key': file_bytes},       # ...
             json={'key': {'key': 'value'}})  # ...

requests.delete(url='http://<url>/<route>')
```

## Valider la connexion

In [5]:
response = requests.get(f'{URL}')
print('Code HTTP:', response.status_code)
pprint(response.json())

Code HTTP: 200
{'message': 'Bonjour et bienvenue au workshop sur le Web!'}


## 1. Création d'un compte
- Soumetter votre IDUL via un POST à l'adresse `http://<url>/account`.
- Vous pouvez soumettre votre idul de cette façon : `{'idul': 'votre-idul'}`.
- Regardez le code HTTP (`response.status_code`) ainsi que le message (`response.text` ou `response.json()`).

## 2. Authentification

- Génial! Vous avez maintenant un compte.
- Ce n'est pas n'importe qui qui peut récupérer des données médicales. Pour pouvoir les récupérer, vous devez être identifié.
- Pour l'identification, il est commun que les API REST utilise un jeton, ou `token` en anglais.

- Pour récupérer un token (donc s'identifier), faîtes un POST à l'adresse `http://<url>/auth`, tout en fournissant votre IDUL, comme à la création du compte.
- Observez la réponse, votre `token` devrait s'y trouver!

## 3. Récupération des données

- Vous avez maintenant votre `token` pour vous authentifier, il est maintenant possible de récupérer les données.
- L'adresse où les données sont accessibles est `http://<url>/data`.
- Faîtes un GET, avec votre token en paramètre (`params={'token': '<votre-token>'}`) pour vérifier quelles données sont accessibles.

- Vous devriez observer 4 noms de donnée. Chacune d'entre elles correspondent à un fichier DICOM de structure (RTStruct), ou de dose (RTDose).
- Pour les récupérer, faîtes un GET à `http://<url>/data/<nom-donnée>`, toujours en ayant votre token en paramètre.
- Remarquez que le contenu de la réponse est un fichier binaire (`response.content`).
    - On ne peut évidemment pas le lire en tant que texte ou JSON. Une fonction vous est fournie pour lire ce contenu et le transformer en `Dataset` (format pour manipuler des fichiers DICOM) : 
        - `dataset = read_dicom_dataset(response.content)`


- Récupérer les 4 fichiers DICOM, et transformez les tous en `Dataset`.

- Maintenant que vous avez les 4 `Dataset`, vérifiez s'ils contiennent des informations personnelles.
    - Observez l'attribut `dataset.PatientName` pour voir s'il contient un nom.

## 4. Anonymisation des données
- En santé, il est important de manipuler des données anonymes lors des activités de recherche.
- Il est donc nécessaire d'anonymiser vos données.
- Pour anonymisé un `Dataset`
    1. Transformez le en `bytes` avec la fonction `dicom_dataset_to_bytes`
    2. Faire un POST à `http://<url>/anonymize` avec le dataset en bytes en tant que fichier
        - `requests.post(..., files={'file': dataset_bytes})`
    3. Récupérer la réponse en dataset (`read_dicom_dataset(response.content)`)

- Vérifiez si les `Dataset` ont toujours de l'informations personnelles (`dataset.PatientName`).

## 5. Calculer des DVH

- Maintenant que vous avez des données anonymes, vous pouvez obtenir les DVH.
- Pour chaque structure :
    1. Transformez les datasets de structure et de dose en `bytes` avec la fonction `dicom_dataset_to_bytes`
    2. Faîtes un POST à `http://<url>/dvh`, les dataset en bytes en tant que fichiers `{'file_struct': struct_bytes, 'file_dose': dose_bytes}`
    3. Récupérez les réponses sous le format JSON (`response.json()`).

## 6. Tracer des DVH

- Maintenant que vous avez les données des DVH, vous pouvez tracer un graphique.
- Pour ce faire, faîtes un POST à `http://<url>/dvh/plot`.
- Le post doit contenir les informations suivantes :
```python
requests.post(
    ...,
    json={
        'dvhs': [
            {'name': ..., 'volumes': ..., 'doses': ..., 'volume_units': ..., 'dose_units': ...},
            {'name': ..., 'volumes': ..., 'doses': ..., 'volume_units': ..., 'dose_units': ...}
            ...
        ]
    }
)
```

- Le contenu de la réponse est un fichier `.png`. Vous pouvez l'afficher avec la fonction `show_image_from_bytes(response.content)`. 

## Voilà, vous avez obtenu des DVH!

Maintenant que vous savez manipuler une API REST, vous pouvez allez voir comment en faire une. 
Cet atelier utilise ce projet : https://github.com/ulaval-rs/prog-fest-web-workshop/blob/main/app.py.
La librairie utilisée pour construire l'application est celle-ci https://flask-restful.readthedocs.io/en/latest/quickstart.html.